# <center>CryptoCurrencies CA#2<center>
<h1><center>Transacting on Bitcoin TestNet / Firing up an Ethereum Node<center></h1>

#### <center>"Ali - Hamzehpour"<center>
#### <center>"810100129"<center>

In [ ]:
!pip install base58 ecdsa pycryptodome python-bitcoinlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.5 MB/s eta 0:00:00


## Address Generation

### Question 1: Base58 and WIF address

Difference Between Testnet and Mainnet Addresses

Mainnet (Main Network):

* The main Bitcoin network where actual bitcoins are transacted.

* Addresses usually start with 1 for legacy addresses (P2PKH) or 3 for multi-signature addresses (P2SH).

* The prefix for WIF private keys is \x80.

* The network byte for addresses is \x00.

Testnet (Test Network):

* A separate Bitcoin network used for testing and development, where the bitcoins have no real-world value.

* Addresses usually start with m or n for legacy addresses (P2PKH) or 2 for multi-signature addresses (P2SH).

* The prefix for WIF private keys is \xef.

* The network byte for addresses is \x6f.

In [ ]:
import hashlib
import base58
import ecdsa
import secrets
from Crypto.Hash import RIPEMD160

def generate_private_key():
    return secrets.token_bytes(32)

def private_key_to_public_key(private_key, compressed=False):
    signing_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
    verifying_key = signing_key.verifying_key
    if compressed:
        public_key = b"\x02" + verifying_key.to_string()[:32] if verifying_key.pubkey.point.y() % 2 == 0 else b"\x03" + verifying_key.to_string()[:32]
    else:
        public_key = b"\x04" + verifying_key.to_string()
    return public_key

def private_key_to_wif(private_key, compressed=False, testnet=True):
    prefix = b'\xef' if testnet else b'\x80'
    extended_key = prefix + private_key
    if compressed:
        extended_key += b'\x01'

    checksum = hashlib.sha256(hashlib.sha256(extended_key).digest()).digest()[:4]
    wif_key = base58.b58encode(extended_key + checksum)

    return wif_key.decode('utf-8')

def ripemd160(data):
    h = RIPEMD160.new()
    h.update(data)
    return h.digest()

def public_key_to_address(public_key, testnet=True):
    sha256_pub = hashlib.sha256(public_key).digest()
    ripemd160_pub = ripemd160(sha256_pub)

    network_byte = b'\x6f' if testnet else b'\x00'
    network_pubkey_hash = network_byte + ripemd160_pub

    checksum = hashlib.sha256(hashlib.sha256(network_pubkey_hash).digest()).digest()[:4]
    binary_address = network_pubkey_hash + checksum
    address = base58.b58encode(binary_address)

    return address.decode('utf-8')

def generate_address():
    private_key = generate_private_key()
    public_key = private_key_to_public_key(private_key, compressed=True)
    wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
    bitcoin_address = public_key_to_address(public_key, testnet=True)
    return bitcoin_address



In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): 1088a616d080d9a011cbbe71f5d9637bccba2962177d15bd0bab32aea796ac27
Public Key (hex): 02137c753c148faf2a645d5e571c967e41087898efa7bb85ce3e2b15cd8e5dc8d6
WIF Private Key: cN8qjuQEeKvmi2E7FtiMtFun45XhoGJekFaP5cx6cH93ch5cN9X1
Bitcoin Testnet Address: n1FnpZB44jVSof8sWzDJo4meqQX4eoBY6x


### Question 2: Vanity Address

In [ ]:
def generate_vanity_address(chars):
    while True:
        address = generate_address()
        if address[1:len(chars) + 1] == chars:
            return address


In [ ]:
generate_vanity_address("4dc")

'n4dcJ6KbV9ajdezmmwT7GtF8p3B8QrWWCS'

## Part 2: Transacting on Bitcoin TestNet

In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): 50246dc6ec7098755edd99b25b94eda72c66567f0fcc2c8a35b9b3af2d70773a
Public Key (hex): 03b1ebe63f21a50f1bec6e1cf2ca2f20acd78937b92b79ff3882a3ee93b81f8f4c
WIF Private Key: cQGVFD8mUrouH5zKpc7BV4ZfQESdCvaMFPkHSTBLg64bZpfJDeUc
Bitcoin Testnet Address: mg9TrmfTpn9ohQ4B9EZv89xrWGemg6CJNX


### Utility Functions:

In [ ]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x

bitcoin.SelectParams("testnet")

def P2PKH_scriptPubKey(key):
    return [OP_DUP, OP_HASH160, Hash160(key), OP_EQUALVERIFY, OP_CHECKSIG]

def P2PKH_scriptSig(txin, txout, txin_scriptPubKey, private_key, i =0):
    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, private_key, i)
    public_key = private_key.pub
    return [signature, public_key]

def get_txout_scriptPubKeys(amount_to_send1, amount_to_send2):
    txout1_scriptPubKey = [OP_TRUE]
    txout2_scriptPubKey = [OP_FALSE]
    txout1 = create_txout(amount_to_send1, txout1_scriptPubKey)
    txout2 = create_txout(amount_to_send2, txout2_scriptPubKey)
    return txout1, txout2

In [ ]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH


def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin = create_txin(txid_to_spend, utxo_index)
    new_tx = create_signed_transaction(txin, txout, txin_scriptPubKey, txin_scriptSig)
    return broadcast_transaction(new_tx)


def create_txin(txid, utxo_index):
    return CMutableTxIn(COutPoint(lx(txid), utxo_index))


def create_txout(amount, scriptPubKey):
    return CMutableTxOut(amount*COIN, CScript(scriptPubKey))


def create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, seckey, i=0):
    tx = CMutableTransaction(txin, txout)
    sighash = SignatureHash(CScript(txin_scriptPubKey), tx,
                            i, SIGHASH_ALL)
    sig = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return sig


def create_signed_transaction(txins, txouts, txin_scriptPubKey, txin_scriptSigs):
    tx = CMutableTransaction(txins, txouts)
    for i, txin in enumerate(txins):
        txin.scriptSig = CScript(txin_scriptSigs[i])
        VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey[i]), tx, i, (SCRIPT_VERIFY_P2SH,))
    return tx


def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction,
    )


### Question 1: One input and two outputs transaction

First we generate the private key and the address for this part:

In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): 3e318f43131189270a55b56e914e12ffa8d80bec79016d5dda1239913cc3425d
Public Key (hex): 0279a9f2c5a86c0c448d95de6653ef42de664a22477f7ef6025912190a5a831084
WIF Private Key: cPfbdSfV5tdBADLRdxWjSJZ9zTxJLUakoSyFKcBDFYPCsATEq9es
Bitcoin Testnet Address: mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm


Then we get some coint from a faucet for this address:

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/f61a531342e14ee16b562d6c8b48157717cba3ad95b241fba53e13d55d4f2b06/)

<p align="center">
    <img src="screenshots/q1-1.png" alt="Image Description">
</p>

Now we make transaction with one input and two outputs where one of the outputs is spendable for every one and the other one is unspendable. The way we make the output unspendable is by using the OP_FALSE opcode which pushes a 0 to the stack and then fails the script. This way the output is unspendable. The way we make the output spendable is by using the OP_TRUE opcode which pushes a 1 to the stack and then the script is valid. This way the output is spendable.

In [ ]:
pv_key = "cPfbdSfV5tdBADLRdxWjSJZ9zTxJLUakoSyFKcBDFYPCsATEq9es"
my_private_key = bitcoin.wallet.CBitcoinSecret(pv_key)
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

def send_from_P2PKH_transaction(amount_to_send1, amount_to_send2, txid_to_spend, utxo_index):
    txout1, txout2 = get_txout_scriptPubKeys(amount_to_send1, amount_to_send2)
    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig([txin], [txout1, txout2], txin_scriptPubKey, my_private_key)
    new_tx = create_signed_transaction([txin], [txout1, txout2], [txin_scriptPubKey], [txin_scriptSig])
    return broadcast_transaction(new_tx)

In [ ]:
all_money = 0.00019566
amount_to_send1 = 0.00000565
amount_to_send2 = 0.00000001
txid_to_spend = ('f61a531342e14ee16b562d6c8b48157717cba3ad95b241fba53e13d55d4f2b06')
utxo_index = 1

print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send1, amount_to_send2, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm
my public key:  0279a9f2c5a86c0c448d95de6653ef42de664a22477f7ef6025912190a5a831084
my private key:  3e318f43131189270a55b56e914e12ffa8d80bec79016d5dda1239913cc3425d01
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "a4de5ac7051f8027e5670419258b799e8e3a3ab35488d423dddff501e07bb758",
    "addresses": [
      "mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm"
    ],
    "total": 566,
    "fees": 16963,
    "size": 178,
    "vsize": 178,
    "preference": "medium",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T18:14:00.393702824Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 2,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "f61a531342e14ee16b562d6c8b48157717cba3ad95b241fba53e13d55d4f2b06",
        "output_index": 1,
        "script": "483045022100e62d6cc680cb426832f72477570c2255e16e935daada76443ca10

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/a4de5ac7051f8027e5670419258b799e8e3a3ab35488d423dddff501e07bb758/)
<p align="center">
    <img src="screenshots/q1-2.png" alt="Image Description">
</p>

Now we spend the spendable output and returns it to our address.

In [ ]:
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin_scriptPubKey = [OP_TRUE]
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = []

    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])

    return broadcast_transaction(new_tx)

In [ ]:
all_money = 0.00000565
amount_to_send = 0.00000060
txid_to_spend = ('a4de5ac7051f8027e5670419258b799e8e3a3ab35488d423dddff501e07bb758')
utxo_index = 0

print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm
my public key:  0279a9f2c5a86c0c448d95de6653ef42de664a22477f7ef6025912190a5a831084
my private key:  3e318f43131189270a55b56e914e12ffa8d80bec79016d5dda1239913cc3425d01
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "149f8eeffc840d9f463b280443cd8318dd45a4c2f26abe173c4acaa1a11a0e54",
    "addresses": [
      "mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm"
    ],
    "total": 60,
    "fees": 505,
    "size": 85,
    "vsize": 85,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T18:15:50.925987274Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "a4de5ac7051f8027e5670419258b799e8e3a3ab35488d423dddff501e07bb758",
        "output_index": 0,
        "output_value": 565,
        "sequence": 4294967295,
        "script_type": "unknown

#### [transaction link](https://live.blockcypher.com/btc-testnet/address/mgAPDPnojeh4pi4YjJoeE6yeRwbP9jqqZm/)

<p align="center">
    <img src="screenshots/q1-3.png" alt="Image Description">
</p>

### Question 2: Pay to MultiSig transaction (P2MS)


In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): c3719a0162a5c33927b0ae8b79d733dfb8bae653f4e62663019c0f37ddd9af68
Public Key (hex): 0259778155081d45164f1473a48b23ff81adce8fa0c21a0498b43d8dda542bd373
WIF Private Key: cU8cqmjffR76VMuPAo3rDtwgjzWHpRJT71pxvqzAb7bP7HcSYjyb
Bitcoin Testnet Address: moFQa4qRz6EWorB4s3Zph7ZFiUSmWz3r6J


Again we create a new private key and new address and gets coin for it:

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/75abd9864fdd93bd508e82d47b0de3bb3c69f7688282f3ac7a52b4b469990ecb/)

<p align="center">
    <img src="screenshots/q2-1.png" alt="Image Description">
</p>

Then we create 3 private addresses:

In [ ]:
pv1 = private_key_to_wif(generate_private_key(), compressed=True, testnet=True)
pv2 = private_key_to_wif(generate_private_key(), compressed=True, testnet=True)
pv3 = private_key_to_wif(generate_private_key(), compressed=True, testnet=True)
print(pv1)
print(pv2)
print(pv3)

cSCAgHQhaeitHx7riSVWyqcdzZsfWAgmvt1KQUMjmsezvVZK7KoM
cUwcBAHL8BgGEmUo5PZgz6KwdkFMFKtKa4KtDRhXrthvuCugXu22
cTnq893hPQknctLPHe71yRgyNcHrj34uc48r7EtPgrqGAiVVwfgQ


The function send_from_P2PKH_transaction is designed to create a multi-signature (multi-sig) Bitcoin transaction. In this context, a multi-sig transaction requires multiple signatures from different private keys to be spent. The script for the transaction output (txout_scriptPubKey) is constructed using the `OP_CHECKMULTISIG` opcode, which specifies that signatures from at least two of the three provided public keys (public_key1, public_key2, public_key3) are needed to spend the output. This is achieved by including the `OP_2` opcode (indicating that two signatures are required) followed by the three public keys and ending with the OP_3 opcode (indicating that three public keys are involved) and `OP_CHECKMULTISIG`. The input (txin) is created from the specified UTXO (txid_to_spend and utxo_index), and its unlocking script (txin_scriptSig) is generated by signing the transaction with the sender's private key (my_private_key). The transaction is then constructed and signed before being broadcast to the Bitcoin network. This setup ensures that the output can only be spent if at least two of the specified private keys provide valid signatures, enhancing the security of the transaction by requiring multiple approvals.

In [ ]:
pv_key = "cU8cqmjffR76VMuPAo3rDtwgjzWHpRJT71pxvqzAb7bP7HcSYjyb"


my_private_key = bitcoin.wallet.CBitcoinSecret(pv_key)
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)
print(my_address)

private_key1 = bitcoin.wallet.CBitcoinSecret(pv1)
private_key2 = bitcoin.wallet.CBitcoinSecret(pv2)
private_key3 = bitcoin.wallet.CBitcoinSecret(pv3)

public_key1 = private_key1.pub
public_key2 = private_key2.pub
public_key3 = private_key3.pub

def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout_scriptPubKey = [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG]
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig([txin], [txout], txin_scriptPubKey, my_private_key)
    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])
    return broadcast_transaction(new_tx)

moFQa4qRz6EWorB4s3Zph7ZFiUSmWz3r6J


In [ ]:
all_money = 0.00018707
amount_to_send = 0.00007715
txid_to_spend = ('096b652fa963eae6d783b5c90fd4eb5a4faa1f245c927d7a74bcd4115ecf8306')
utxo_index = 1

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "7b0522b5622d9252a0056408ec2eb8b32ee6dfb4b89b58d3c1d0d5c036499154",
    "addresses": [
      "moFQa4qRz6EWorB4s3Zph7ZFiUSmWz3r6J",
      "zWC8iLDhSpH8JnbfTFTJRNpYjXF24Bqb4t"
    ],
    "total": 7715,
    "fees": 10992,
    "size": 272,
    "vsize": 272,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T20:17:48.371964575Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "096b652fa963eae6d783b5c90fd4eb5a4faa1f245c927d7a74bcd4115ecf8306",
        "output_index": 1,
        "script": "483045022100fb88906a17157e02dcc5199848ac71525b900064879fb4d3442d8788e88ce860022070c36242311e337b3bc6697e101af97c012a54b755e1095982713ecf7796fcd601210259778155081d45164f1473a48b23ff81adce8fa0c21a0498b43d8dda542bd373",
        "output_value

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/7b0522b5622d9252a0056408ec2eb8b32ee6dfb4b89b58d3c1d0d5c036499154/)

<p align="center">
    <img src="screenshots/q2-2.png" alt="Image Description">
</p>

Now we spend it by providing two signatures:

In [ ]:
def scriptSig(txin, txout, txin_scriptPubKey):
    signature1 = create_OP_CHECKSIG_signature(txin[0], txout, txin_scriptPubKey, private_key1)
    signature2 = create_OP_CHECKSIG_signature(txin[0], txout, txin_scriptPubKey, private_key2)
    return [OP_0, signature1, signature2]

def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    txin_scriptPubKey = [OP_2, public_key1, public_key2, public_key3, OP_3, OP_CHECKMULTISIG]
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = scriptSig([txin], [txout], txin_scriptPubKey)

    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])

    return broadcast_transaction(new_tx)


In [ ]:
all_money = 0.00017715
amount_to_send = 0.00000005
txid_to_spend = ('d9779aefbbe106b7d27810ac5d3a9d47082c9d3577ed4f1a20e4db490c0f2faa')
utxo_index = 0

print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mxtM4YfiT8HAoitbQueeEQv9B7zmWyk9Ds
my public key:  03bc156be04913daa388c7204a9823a15f4f66cdd6f9d0e940ce09a6194ca1134c
my private key:  04a1668fe873e5fd74a275e0186f3cad34a9cba9e6ef228d5e02eb7790d8c18e01
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "5b996e6f90540f13ede51f0aa5cd72a159791041bdf29663e3927dd3195aa19f",
    "addresses": [
      "zCy2Bg7BUf4TCk68jVswFLPpEvUzCar1ci",
      "mxtM4YfiT8HAoitbQueeEQv9B7zmWyk9Ds"
    ],
    "total": 5,
    "fees": 17710,
    "size": 231,
    "vsize": 231,
    "preference": "medium",
    "relayed_by": "34.106.119.192",
    "received": "2024-05-29T07:33:56.008906306Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "d9779aefbbe106b7d27810ac5d3a9d47082c9d3577ed4f1a20e4db490c0f2faa",
        "output_index": 0,
        "script": "00483045022100e853b6dc

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/e60d565b68e5974227f6ccb0c7c600d44582ebe53d8140a480a588b7621c2ad0/)

<p align="center">
    <img src="screenshots/q2-3.png" alt="Image Description">
</p>

**2.3**: Custom Transaction (BitCoin Scripting)


Again we create a new address and get the coin:

In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): 261a7b24b4122757251a7882a99ba9b68674448bd8d9e169106e570d4872b48e
Public Key (hex): 02897ad0f50681a5eada729ffe28da50a351479100beb99b4ba83df7b8134bcf92
WIF Private Key: cNrmbhRoY4RFFxbop9w4avrTZTmH1NHhLik5kEUHVk5h3eQ8LPSX
Bitcoin Testnet Address: mjjtHVT4DmR11GH8rUC6r9obR42x9ssDV8


#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/6059c87cdf13dc8feac75de48bc2cf9abdbb3af8aa9039bfe9d7369f488eadad/)

<p align="center">
    <img src="screenshots/q3-1.png" alt="Image Description">
</p>


The provided txout_scriptPubKey creates a Bitcoin output with a conditional spending mechanism using the `OP_IF ... OP_ELSE ... OP_ENDIF `structure. This script allows the output to be spent in one of two ways: either by fulfilling a numerical condition or by providing a prehashed password. The first part of the script, within the OP_IF block, requires the spender to provide a number such that when subtracted from another number on the stack, the result is greater than 18 (`OP_SUB 18 OP_GREATERTHAN`). If this condition is not met, the script execution jumps to the `OP_ELSE` block, where the spender must provide a value that, when hashed using `OP_HASH160`, matches a predefined hash (password_hash). This dual-condition mechanism enhances the security and flexibility of the transaction by allowing it to be authorized either through a numerical threshold or a password verification.







In [ ]:
import bitcoin.wallet
from bitcoin.core import COIN, b2lx, serialize, x, lx, b2x

pv_key = "cNrmbhRoY4RFFxbop9w4avrTZTmH1NHhLik5kEUHVk5h3eQ8LPSX"
my_private_key = bitcoin.wallet.CBitcoinSecret(pv_key)
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

def P2PKH_scriptPubKey(key):
    return [OP_DUP, OP_HASH160, Hash160(key), OP_EQUALVERIFY, OP_CHECKSIG]

def P2PKH_scriptSig(txin, txout, txin_scriptPubKey):
    signature = create_OP_CHECKSIG_signature(txin, txout, txin_scriptPubKey, my_private_key)
    return [signature, my_public_key]

def get_txout_scriptPubKeys(amount_to_send):

    PASSWORD = 810100129
    password_hash = Hash160((PASSWORD).to_bytes(30, byteorder="little"))
    txout_scriptPubKey = [OP_IF, OP_SUB, 18, OP_GREATERTHAN, OP_ELSE, OP_HASH160, password_hash, OP_EQUAL, OP_ENDIF]
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    return txout

def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout = get_txout_scriptPubKeys(amount_to_send)
    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig([txin], [txout], txin_scriptPubKey)
    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])
    return broadcast_transaction(new_tx)

In [ ]:
all_money = 0.00015129
amount_to_send = 0.00005129
txid_to_spend = ('19ccae138a5cb45747816535b9d63fc99c9b1e1ce54de782aba2f386799ba5d3')
utxo_index = 0
print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mjjtHVT4DmR11GH8rUC6r9obR42x9ssDV8
my public key:  02897ad0f50681a5eada729ffe28da50a351479100beb99b4ba83df7b8134bcf92
my private key:  261a7b24b4122757251a7882a99ba9b68674448bd8d9e169106e570d4872b48e01
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "630b771a5d9a6043c229d7edc148d46ec93007e56fef2e98076f4ff79f2bfcf6",
    "addresses": [
      "mjjtHVT4DmR11GH8rUC6r9obR42x9ssDV8"
    ],
    "total": 5129,
    "fees": 10000,
    "size": 196,
    "vsize": 196,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T18:38:10.181105864Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "19ccae138a5cb45747816535b9d63fc99c9b1e1ce54de782aba2f386799ba5d3",
        "output_index": 0,
        "script": "47304402205dd91a92c48e56dca0797deb3e7b6ec5ce81208c76e78de4a743bc18f

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/630b771a5d9a6043c229d7edc148d46ec93007e56fef2e98076f4ff79f2bfcf6/)

<p align="center">
    <img src="screenshots/q3-2.png" alt="Image Description">
</p>

Now in order to spend it using password we give `[(PASSWORD).to_bytes(30, byteorder="little"), OP_FALSE]` as the script sig.

In [ ]:
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    PASSWORD = 810100129
    password_hash = Hash160((PASSWORD).to_bytes(30, byteorder="little"))

    txin_scriptPubKey = [OP_IF, OP_SUB, 18, OP_GREATERTHAN, OP_ELSE, OP_HASH160, password_hash, OP_EQUAL, OP_ENDIF]
    txin = create_txin(txid_to_spend, utxo_index)

    txin_scriptSig = [(PASSWORD).to_bytes(30, byteorder="little"), OP_FALSE]

    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])

    return broadcast_transaction(new_tx)

In [ ]:
all_money = 0.00005129
amount_to_send = 0.00000129
txid_to_spend = ('630b771a5d9a6043c229d7edc148d46ec93007e56fef2e98076f4ff79f2bfcf6')
utxo_index = 0

print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mjjtHVT4DmR11GH8rUC6r9obR42x9ssDV8
my public key:  02897ad0f50681a5eada729ffe28da50a351479100beb99b4ba83df7b8134bcf92
my private key:  261a7b24b4122757251a7882a99ba9b68674448bd8d9e169106e570d4872b48e01
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "23de93f06b371c06195c4c1995dcef61a3d3e03e45b61cfd8cf7b8c3ef0e008f",
    "addresses": [
      "mjjtHVT4DmR11GH8rUC6r9obR42x9ssDV8"
    ],
    "total": 129,
    "fees": 5000,
    "size": 117,
    "vsize": 117,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T18:39:01.080657405Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "630b771a5d9a6043c229d7edc148d46ec93007e56fef2e98076f4ff79f2bfcf6",
        "output_index": 0,
        "script": "1ea1254930000000000000000000000000000000000000000000000000000000",
  

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/23de93f06b371c06195c4c1995dcef61a3d3e03e45b61cfd8cf7b8c3ef0e008f/)
<p align="center">
    <img src="screenshots/q3-3.png" alt="Image Description">
</p>

We make the transaction again and this time in spending we give birth year and current year to evaluate the age.

In [ ]:
private_key = generate_private_key()
public_key = private_key_to_public_key(private_key, compressed=True)
wif_private_key = private_key_to_wif(private_key, compressed=True, testnet=True)
bitcoin_address = public_key_to_address(public_key, testnet=True)

print(f'Private Key (hex): {private_key.hex()}')
print(f'Public Key (hex): {public_key.hex()}')
print(f'WIF Private Key: {wif_private_key}')
print(f'Bitcoin Testnet Address: {bitcoin_address}')

Private Key (hex): ba6e6436465d1a91d18bf95eaa1ff05c42d3bb885c28d8a4d83c45c2184665f4
Public Key (hex): 03c2e84df5cea5444a7a7953d056aeda123c8446c777e04751aec147418a82fda0
WIF Private Key: cTq6jBjm38BNWyfEfus5oszTCUEfSpwCuJuWcaX3rg4v8q4ZWUvR
Bitcoin Testnet Address: mqwU9YE1Emo7An4PjhmSJbvU5Gqkv3PHR2


#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/72d18bab425a15620f74752a936538d28d1d86d5fafb199f70e5e53916c86f6c/)

<p align="center">
    <img src="screenshots/q3-4.png" alt="Image Description">
</p>

In [34]:
pv_key = "cTq6jBjm38BNWyfEfus5oszTCUEfSpwCuJuWcaX3rg4v8q4ZWUvR"
my_private_key = bitcoin.wallet.CBitcoinSecret(pv_key)
my_public_key = my_private_key.pub
my_address = bitcoin.wallet.P2PKHBitcoinAddress.from_pubkey(my_public_key)

def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout = get_txout_scriptPubKeys(amount_to_send)
    txin_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txin = create_txin(txid_to_spend, utxo_index)
    txin_scriptSig = P2PKH_scriptSig([txin], [txout], txin_scriptPubKey)
    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])
    return broadcast_transaction(new_tx)


all_money = 0.00014099
amount_to_send = 0.00004099
txid_to_spend = ('5f6071c47a323bcafeb6d14bc9ede4791d169e3d525dcd014c5ce5347ff4ecb6')
utxo_index = 1
print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mqwU9YE1Emo7An4PjhmSJbvU5Gqkv3PHR2
my public key:  03c2e84df5cea5444a7a7953d056aeda123c8446c777e04751aec147418a82fda0
my private key:  ba6e6436465d1a91d18bf95eaa1ff05c42d3bb885c28d8a4d83c45c2184665f401
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "72d18bab425a15620f74752a936538d28d1d86d5fafb199f70e5e53916c86f6c",
    "addresses": [
      "mqwU9YE1Emo7An4PjhmSJbvU5Gqkv3PHR2"
    ],
    "total": 4099,
    "fees": 10000,
    "size": 196,
    "vsize": 196,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T19:03:03.831023174Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "5f6071c47a323bcafeb6d14bc9ede4791d169e3d525dcd014c5ce5347ff4ecb6",
        "output_index": 1,
        "script": "473044022058d4ed5bcc4826874a451c6e9dfa221519ebc8135be544afbdab32328

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/72d18bab425a15620f74752a936538d28d1d86d5fafb199f70e5e53916c86f6c/)
<p align="center">
    <img src="screenshots/q3-5.png" alt="Image Description">
</p>

In [35]:
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index):
    txout_scriptPubKey = P2PKH_scriptPubKey(my_public_key)
    txout = create_txout(amount_to_send, txout_scriptPubKey)
    PASSWORD = 810100129
    password_hash = Hash160((PASSWORD).to_bytes(30, byteorder="little"))

    txin_scriptPubKey = [OP_IF, OP_SUB, 18, OP_GREATERTHAN, OP_ELSE, OP_HASH160, password_hash, OP_EQUAL, OP_ENDIF]
    txin = create_txin(txid_to_spend, utxo_index)

    txin_scriptSig = [2024, 2002, OP_TRUE]

    new_tx = create_signed_transaction([txin], [txout], [txin_scriptPubKey], [txin_scriptSig])

    return broadcast_transaction(new_tx)

all_money = 0.00004099
amount_to_send = 00.00000099
txid_to_spend = ('72d18bab425a15620f74752a936538d28d1d86d5fafb199f70e5e53916c86f6c')
utxo_index = 0

print("my address: ", my_address)
print("my public key: ", my_public_key.hex())
print("my private key: ", my_private_key.hex())
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index)
print("response status code: ", response.status_code)
print("reponse reason: ", response.reason)
print("response text: ", response.text)

my address:  mqwU9YE1Emo7An4PjhmSJbvU5Gqkv3PHR2
my public key:  03c2e84df5cea5444a7a7953d056aeda123c8446c777e04751aec147418a82fda0
my private key:  ba6e6436465d1a91d18bf95eaa1ff05c42d3bb885c28d8a4d83c45c2184665f401
response status code:  201
reponse reason:  Created
response text:  {
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "302c291a456990c900fcabd58c3ca3fbfc2ea9bfe381847327351cc4bff64f6c",
    "addresses": [
      "mqwU9YE1Emo7An4PjhmSJbvU5Gqkv3PHR2"
    ],
    "total": 99,
    "fees": 4000,
    "size": 117,
    "vsize": 117,
    "preference": "low",
    "relayed_by": "34.125.143.10",
    "received": "2024-05-29T19:03:40.695512626Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 1,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "72d18bab425a15620f74752a936538d28d1d86d5fafb199f70e5e53916c86f6c",
        "output_index": 0,
        "script": "1ea1254930000000000000000000000000000000000000000000000000000000",
   

#### [transaction link](https://live.blockcypher.com/btc-testnet/tx/302c291a456990c900fcabd58c3ca3fbfc2ea9bfe381847327351cc4bff64f6c/)
<p align="center">
    <img src="screenshots/q3-6.png" alt="Image Description">
</p>

## Questions

### Proof of burn 

Proof of Burn is a consensus mechanism in blockchain where users "burn" their coins (send them to an address that cannot be spent from), in exchange for gaining some form of network credit. Essentially, coins are intentionally removed from circulation and "destroyed" to demonstrate that the user has sacrificed valuable resources to participate in the network.

Economic Impact of Burning Coins: Burning coins can stabilize the value of a cryptocurrency network in several ways:

* Reducing Supply: By permanently removing coins from circulation, the total supply of the cryptocurrency is reduced. This can increase scarcity, potentially increasing the value of the remaining coins if demand remains constant or increases.

* Incentivizing Long-Term Commitment: Participants who burn coins demonstrate a long-term commitment to the network, which can foster trust and stability. This commitment can be particularly important for new or developing networks.

* Counteracting Inflation: Many cryptocurrencies have a continuous issuance of new coins (inflation). Burning coins can help counteract inflationary pressures, thereby helping to maintain the purchasing power of the cryptocurrency.

* Network Participation: Burning coins can be used as a way to gain privileges within the network, such as mining rights, voting power, or other forms of influence. This incentivizes participants to engage with and support the network, contributing to its overall health and stability.


### Bitcoin scripting language

The Bitcoin scripting language, known simply as Bitcoin Script, is a stack-based scripting language used to define the conditions under which a Bitcoin transaction output can be spent. Bitcoin Script is intentionally designed to be simple and limited in functionality to ensure security and predictability.

Is Bitcoin Script Turing Complete?
No, Bitcoin Script is not Turing Complete. A Turing Complete language can perform any computation given enough time and resources, meaning it can implement any algorithm. Bitcoin Script lacks several features required for Turing Completeness, such as loops and more complex control flow structures, to avoid potential security risks like infinite loops or excessive resource consumption.

Advantages of Using a Turing Complete Language:

* Increased Functionality and Flexibility: A Turing Complete language can execute more complex and sophisticated contracts. This would allow for the implementation of advanced features like multi-step transactions, conditional operations, and state-dependent logic.

* Enhanced Decentralized Applications (dApps): With Turing Completeness, it is possible to create more versatile and powerful decentralized applications (dApps) that can perform a wide variety of tasks, from decentralized finance (DeFi) to complex automated processes.

* Smart Contracts: A Turing Complete language allows for the creation of smart contracts that can handle complex business logic and automate processes without the need for intermediaries. Ethereum's Solidity is an example of a Turing Complete language that facilitates such contracts.

However, there are also potential disadvantages:

* Security Risks: Turing Complete languages can be more prone to bugs and vulnerabilities, such as infinite loops or unexpected behaviors, which can be exploited by malicious actors.

* Increased Complexity: The complexity of a Turing Complete language can make it more difficult to audit and verify the correctness and security of scripts, potentially leading to more frequent errors or security breaches.

* Resource Consumption: More complex computations can consume more network resources, leading to higher transaction costs and slower execution times.

## Part 2: Running an Ethereum Node

### Step 1: Install Geth

<p align="center">
    <img src="screenshots/1-1.png" alt="Image Description" width="800" height="400">
</p>

<p align="center">
    <img src="screenshots/1-2.png" alt="Image Description" width="800" height="400">
</p>

<p align="center">
    <img src="screenshots/1-3.png" alt="Image Description" width="800" height="400">
</p>

`geth --help` command:

* NAME: Provides the name of the command-line interface, which is geth in this case.

* USAGE: Specifies how to use the geth command, including options, commands, and arguments.

* VERSION: Displays the version number of the geth software.

* COMMANDS: Lists the available commands that you can use with geth, such as managing accounts, starting an interactive environment, initializing a new genesis block, etc.

* GLOBAL OPTIONS: Enumerates the options that apply globally to the geth command, such as configuration settings, data directory, synchronization mode, network settings, logging verbosity, etc.


### Step 2: Config genesis block

<p align="center">
    <img src="screenshots/2-1.png" alt="Image Description" width="800" height="400">



### Step 3: Creating 3 nodes

<p align="center">
    <img src="screenshots/3-1.png" alt="Image Description" width="800" height="400">

#### **Question 1: What's the use of nodes in network and what does light node and full node mean?**

* Verification: Nodes verify and validate transactions and blocks on the network, ensuring that the transactions adhere to the consensus rules of the blockchain.

* Propagation: Nodes propagate transactions and blocks to other nodes, helping to disseminate new information across the network.

* Consensus: Nodes participate in the consensus mechanism of the blockchain network, whether it's proof-of-work (as in Ethereum's current state) or another consensus algorithm.

* Storage: Nodes store a copy of the entire blockchain ledger, allowing them to provide historical data and support new nodes joining the network.

`Light Node`: Light nodes are designed for users who don't need to store the entire blockchain or participate fully in the network's consensus process.

`Full Node`: Full nodes are integral to the security and decentralization of the network. They maintain a complete copy of the blockchain ledger and actively participate in the consensus process.


### Step 4: Creating accounts and Starting nodes

<p align="center">
    <img src="screenshots/4-1.png" alt="Image Description" width="800" height="400">

<p align="center">
    <img src="screenshots/4-2.png" alt="Image Description" width="800" height="400">

<p align="center">
    <img src="screenshots/4-3.png" alt="Image Description" width="800" height="400">

<p align="center"> 
    <img src="screenshots/4-4.png" alt="Image Description" width="800" height="400">

#### Step 5: Initializing Nodes and Running them

<p align="center">
    <img src="screenshots/5-1.png" alt="Image Description" width="800" height="400">

<p align="center">
    <img src="screenshots/5-2.png" alt="Image Description" width="800" height="400">

<p align="center">
    <img src="screenshots/5-3.png" alt="Image Description" width="800" height="400">

<p align="center">  
    <img src="screenshots/5-4.png" alt="Image Description" width="800" height="400">

<p align="center">  
    <img src="screenshots/5-5.png" alt="Image Description" width="800" height="400">

### Step 6: Connecting to a node

<p align="center">
    <img src="screenshots/6-1.png" alt="Image Description" width="800" height="400">


### Step 7: Connecting nodes to each other

<p align="center">
    <img src="screenshots/7-1.png" alt="Image Description" >

For The main node:

<p align="center">
    <img src="screenshots/7-2.png" alt="Image Description" >

For other nodes:

<p align="center">
    <img src="screenshots/7-3.png" alt="Image Description">

### Step 8: Creating a transaction

<p align="center">
    <img src="screenshots/8-1.png" alt="Image Description">

<p align="center">
    <img src="screenshots/8-2.png" alt="Image Description">

<p align="center">
    <img src="screenshots/8-3.png" alt="Image Description">

<p align="center">
    <img src="screenshots/8-4.png" alt="Image Description">

#### **Question 2: Explain the logs in the mining process**

Explanation of Log Entries:

`INFO [05-28|02:35:53.848] Mined potential block                  number=22 hash=316890..0700ee`

* [05-28|02:35:53.848]: Timestamp indicating when the event occurred.
* Mined potential block: Message indicating a potential block has been mined.
* number=22: The block number.
* hash=316890..0700ee: The hash of the block.

`INFO [05-28|02:35:53.848] Mined potential block                  number=22 hash=316890..0700ee`

Indicates that a new block has been mined and is a candidate to be added to the blockchain.
The block number and hash are provided for reference.
Commit New Sealing Work:


`INFO [05-28|02:35:53.851] Commit new sealing work `

Indicates that the mining process is starting new work on sealing (mining) the next block.
Successfully Sealed New Block:


`INFO [05-28|02:35:53.851] Successfully sealed new block           number=23 sealhash=7a58b8..db0828`

Indicates that the block has been successfully sealed (mined).
The block number and seal hash are provided.
Block Reached Canonical Chain:


`INFO [05-28|02:35:53.851] "block reached canonical chain"       number=16 hash=25ecb9..00baad`

Indicates that the mined block has been accepted into the main blockchain (canonical chain).
The block number and hash are provided.

`INFO [05-28|02:35:54.199] Generating DAG in progress epoch=1 percentage=72 elapsed=2m34.219s`

Indicates that the Directed Acyclic Graph (DAG) is being generated for the mining process.
The epoch, percentage of completion, and elapsed time are provided.


`elapsed=882.594ms`

Indicates the time taken for a specific action, in this case, the time taken to reach the canonical chain.

From the logs We can say the mining process is like this:

1. Mined Potential Block: A potential block is mined and considered for addition to the blockchain.

2. Commit New Sealing Work: The miner starts working on the next block.

3. Successfully Sealed New Block: The mined block is successfully sealed, meaning it is ready to be added to the blockchain.

4. Block Reached Canonical Chain: The block is accepted into the canonical chain.

5. Generating DAG: Periodically, the mining process involves generating a new DAG file, which is essential for the Ethash proof-of-work algorithm used by Ethereum.

#### **Question 3: Since 2015, up to now, 19722000 blocks have been mined in the main Ethereum network. Suppose that as an exercise as an attacker, we quickly mine up to block 19722000 in our local blockchain, and then attempt to broadcast it to the network, claiming that our local blockchain is the true main Ethereum blockchain! Can we do such a thing? If yes, explain the circumstances under which this would succeed, and if not, explain why it would fail.**

No, it is not possible to successfully broadcast a local blockchain as the true main Ethereum blockchain. This is because the Ethereum network operates on a consensus mechanism that requires a majority of nodes to agree on the validity of blocks and transactions. Because other full nodes on the network have a copy of the entire blockchain and are actively participating in the consensus process, they would reject the local blockchain as it does not match the network's version of the blockchain.

Also we won't have the computational power to mine all the blocks up to block 19722000 in our local blockchain. The main Ethereum network has a large number of miners and nodes, making it practically impossible for a single entity to mine all the blocks and overtake the network.

Also We don't broadcast the whole blockchain, we broadcast the blocks we mined and the network will verify the blocks and if they are valid, other nodes will add them to their blockchain so broadcasting all those blocks is practically impossible.